# Imports and Constants

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *

C:\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python38\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Python38\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
from_who = "mohameds"   # 2 possibilities : "mohameds" or "our"
maps_folder="brain_maps/"+from_who+"_maps"
masks_folder="masks/"+from_who+"_masks"

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
class_labels = [""] * (2 * length_one_modality)
for i in range(length_one_modality * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length_one_modality])
labels["aud"] = np.array(class_labels[length_one_modality:])
labels_same = labels["vis"]

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                           (["vis"], ["PT_L", "PT_R"]),
                           (["aud"], ["V5_L", "V5_R"]),
                           (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                            (["vis", "aud"], ["PT_L", "PT_R"])]

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = 'linear', random_state=SEED)  # other parameters are default parameters
pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])
decoder = Decoder(n_perm=n_individual_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)

# Loading data

In [3]:
maps_masked, masks_exist = load_full_data(subjects_ids, length_one_modality, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))
decoder.set_masks_exist(masks_exist)

# Within-modality decoding

In [4]:
confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
group_confusion_matrixes_within = average_dicos(confusion_matrixes_within)

# Cross-modal decoding

In [5]:
confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
group_confusion_matrixes_cross = average_dicos(confusion_matrixes_cross)

# Bootstrapping to assess group-level significance

In [6]:
n_single_perm = 100
n_bootstrap = 100_000

In [7]:
within_cf_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,within_modal_tasks_regions,maps_masked,n_subjects, within_modality=True)
within_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, within_cf_100_perm)
bootstrapped_distribution_within = compute_bootstrap_distribution(n_bootstrap, n_subjects, within_scores_100_perm, n_single_perm)

Running models done in 57.1590359210968 seconds
Running models done in 1.6309659481048584 seconds


In [8]:
cross_cfm_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,cross_modal_tasks_regions,maps_masked,n_subjects, within_modality=False)
cross_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, cross_cfm_100_perm)
bootstrapped_distribution_cross = compute_bootstrap_distribution(n_bootstrap, n_subjects, cross_scores_100_perm, n_single_perm)

Running models done in 58.86799192428589 seconds
Running models done in 1.0140414237976074 seconds


# Saving results

In [9]:
params = classifier.get_params()
out_directory = "out/"+from_who+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
create_directory(out_directory)

In [10]:
save_dicts(out_directory+"masks_exist.csv", masks_exist, list(masks_exist[0].keys()), subjects_ids)
save_dicts(out_directory+"confusion_matrixes_within.csv", confusion_matrixes_within, list(confusion_matrixes_within[0].keys()), subjects_ids)
acc_within = compute_metric(out_directory, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, ret = True)
compute_metric(out_directory, subjects_ids, {'name' : 'recall', 'function':recall}, "within", masks_exist)
compute_metric(out_directory, subjects_ids, {'name' : 'precision', 'function':precision}, "within", masks_exist)

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


In [11]:
save_dicts(out_directory+"confusion_matrixes_cross.csv", confusion_matrixes_cross, list(confusion_matrixes_cross[0].keys()), subjects_ids)
acc_cross = compute_metric(out_directory, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "cross", masks_exist, ret = True)
compute_metric(out_directory, subjects_ids, {'name' : 'recall', 'function':recall}, "cross", masks_exist)
compute_metric(out_directory, subjects_ids, {'name' : 'precision', 'function':precision}, "cross", masks_exist)

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


In [12]:
within_modality_group_results = average_dicos(acc_within)
save_dicts(out_directory+"group_scores_within.csv", [within_modality_group_results], list(within_modality_group_results.keys()), [0])

cross_modality_group_results = average_dicos(acc_cross)
save_dicts(out_directory+"group_scores_cross.csv", [cross_modality_group_results], list(cross_modality_group_results.keys()), [0])

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


In [13]:
save_dicts(out_directory+"accuracy_bootstraps_within.csv", [bootstrapped_distribution_within], list(bootstrapped_distribution_within[0].keys()), range(n_bootstrap))

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


In [14]:
save_dicts(out_directory+"accuracy_bootstraps_cross.csv", [bootstrapped_distribution_cross], list(bootstrapped_distribution_cross[0].keys()), range(n_bootstrap))

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


# Plotting results (from files of saved results)

In [15]:
params = classifier.get_params()
results_directory = "out/"+from_who+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"     # to be changed depending on where you want to see the results from
cv_group_df = retrieve_cv_metric(results_directory, "group_scores")
cv_df = retrieve_cv_metric(results_directory, "accuracy")
cfm_df = retrieve_cv_matrixes(results_directory)

In [16]:
# compute estimated p-values for bootstrap
bootstrap_df  = retrieve_bootstrap_metric(results_directory, "accuracy")
pvals = compute_p_val_bootstrap(bootstrap_df, cv_group_df)
save_dicts(out_directory+"estimated_pval_bootstrap.csv", [pvals], list(pvals.keys()), [0])

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\savingOutputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)


In [17]:
plt_directory = "plots/"+from_who+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)
plotter.plot_cv_score_with_points(cv_df, chance_level = True)

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\utility.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vb_df = vb_df.append(new_entry, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\utility.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vb_df = vb_df.append(new_entry, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\utility.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vb_df = vb_df.append(new_entry, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\utility.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vb_df = vb_df.append(new_entry,

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 1656x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [18]:
plotter.plot_bootstrap(bootstrap_df, cv_group_df, pvals, 30)
group_cfm = compute_group_confusion_matrix(cfm_df, subjects_ids)
plotter.plot_group_confusion_matrix(group_cfm, classes)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi